# Multiple inheritance

What if we wanted to add a second kind of loss in the forest, for instance a tree-killing mold that can develop if the density of trees is too high.  It might be kind of interesting see the interactions between two different kinds of growth-limiting phenomena.

First, let's go back to our simple `Leaf` example.  If we create two different subclasses of `Leaf`, one which provides a `change()` method, and one which provides a `fall()` method.  A subclass that inherits from both classes will have both methods.

In [ ]:
class Leaf(object):
    def __init__(self, color="green"):
        self.color = color

class ColorChangingLeaf(Leaf):
    def change(self, new_color="brown"):
        self.color = new_color

class DeciduousLeaf(Leaf):
    def fall(self):
        print "Plunk"
        
class MapleLeaf(DeciduousLeaf, ColorChangingLeaf):
    pass

Now we see that an instance of `MapleLeaf` will inherit the behaviors of both of its parents:

In [ ]:
leaf = MapleLeaf()
leaf.change("yellow")
print leaf.color
leaf.fall()

Now that we've seen a simple example of multiple inheritance, let's go back to our forest example.

We know that we could make another kind of Forest by subclassing `Forest`, so let's do that.

While we're doing that, I want to introduce a minor refactoring of the Forest to create a `losses` attribute that we can use to generically track mold or fires or whatever future agent we can think of.  [If you did the `mold_prone_forest` exercise, this will be familiar.]

In [ ]:
import numpy as np
from matplotlib.pyplot import figure, legend, plot
%matplotlib inline

from scipy.ndimage.measurements import label

class Forest(object):
    """ A Forest can grow but won't burn.
    """

    def __init__(self, size=(150, 150), p_sapling=0.0025):
        self.size = size
        self.trees = np.zeros(self.size, dtype=bool)
        self.p_sapling = p_sapling

    @property
    def name(self):
        return self.__class__.__name__

    @property
    def num_cells(self):
        return self.size[0] * self.size[1]

    @property
    def tree_fraction(self):
        return self.trees.sum() / float(self.num_cells)

    def grow_trees(self):
        growth_sites = self._rand_bool(self.p_sapling)
        self.trees[growth_sites] = True

    def _rand_bool(self, p):
        return np.random.uniform(size=self.trees.shape) < p

    def advance_one_step(self):
        self.grow_trees()

In [ ]:
class MoldProneForest(Forest):
    """Tree-killing mold can grow in tree groves of a certain size.
    """
    def __init__(self, p_mold=3.0e-3, critical_density=3, *args, **kwargs):
        super(MoldProneForest, self).__init__(*args, **kwargs)
        self.p_mold = p_mold
        self.critical_density = critical_density
        self.mold = np.zeros(self.size, dtype=bool)

    @property
    def mold_fraction(self):
        return self.mold.sum() / float(self.num_cells)

    @property
    def losses(self):
        return self.mold

    def advance_one_step(self):
        self.grow_trees()
        self._grow_mold_and_kill_trees()

    def _grow_mold_and_kill_trees(self):
        working_size = (self.size[0] + 2, self.size[1] + 2)
        tmp_trees = np.zeros(working_size, dtype=np.int8)
        tmp_trees[1:-1, 1:-1] = self.trees

        north = tmp_trees[:-2, 1:-1]
        south = tmp_trees[2:, 1:-1]
        east = tmp_trees[1:-1, :-2]
        west = tmp_trees[1:-1, 2:]
        density = north + south + east + west
        mold_prone_trees = density >= self.critical_density

        self.mold = self._rand_bool(self.p_mold) & mold_prone_trees
        self.trees[self.mold] = False

Now we have the `.losses` attribute, which for a `MoldProneForest` is the same as the `.mold` array.  We can similarly set `.losses` to `.fire` in the `BurnableForest` subclasses.   Notice that we defined the `loss_fraction` in `Forest` and because of the interface design, we never have to redefine that attribute, just the definition of the `.losses` attribute.

Conveniently, we only have to redefine `BurnableForest` (and not its subclasses) to take advantage.

In [ ]:
class BurnableForest(Forest):
    """ BurnableForest supports fires.
    """

    def __init__(self, p_lightning=5.0e-6, *args, **kwargs):
        super(BurnableForest, self).__init__(*args, **kwargs)
        self.p_lightning = p_lightning
        self.fires = np.zeros(self.size, dtype=bool)

    @property
    def fire_fraction(self):
        return self.fires.sum() / float(self.num_cells)

    def start_fires(self):
        lightning_strikes = self._rand_bool(self.p_lightning) & self.trees
        self.fires[lightning_strikes] = True

    def burn_trees(self):
        """The rules for burning trees:
        -New fires start in tree-filled adjacent cells.
        -Trees in currently burning cells burn and disappear.

        The fire-spreading algorithm is calculated on an array padded
        around the edges with zeros to allow vectorized computations.
        """
        working_size = (self.size[0] + 2, self.size[1] + 2)
        fires = np.zeros(working_size, dtype=bool)
        fires[1:-1, 1:-1] = self.fires
        north = fires[:-2, 1:-1]
        south = fires[2:, 1:-1]
        east = fires[1:-1, :-2]
        west = fires[1:-1, 2:]
        new_fires = (north | south | east | west) & self.trees
        self.trees[self.fires] = False
        self.fires = new_fires

    def advance_one_step(self):
        self.grow_trees()
        self.start_fires()
        self.burn_trees()

    @property
    def losses(self):
        return self.fires

In [ ]:
class InstantBurnForest(BurnableForest):
    """BurnableForest where fires spread instantaneously relative to the
    growth of new saplings.
    """
    def advance_one_step(self):
        self.grow_trees()
        self._strike_and_burn()

    def _strike_and_burn(self):
        strikes = self._rand_bool(self.p_lightning) & self.trees
        groves, num_groves = label(self.trees)
        fires = set(groves[strikes])
        self.fires.fill(False)
        for fire in fires:
            self.fires[groves == fire] = True
        self.trees[self.fires] = False

In [ ]:
tree_history = []
forests = [
    Forest(),
    MoldProneForest(),
    InstantBurnForest(),
]
num_generations = 2500
for i in xrange(num_generations):
    for forest in forests:
        forest.advance_one_step()
    tree_history.append(tuple(f.tree_fraction for f in forests))

In [ ]:
p = plot(tree_history)

Notice the effect of the mold is to limit the maximum density of the forest, and `p_mold` determines the level of variability.

In Python, we can use multiple inheritance to combine the features of both kinds of forest with minimal further coding.  We do this by specifying more than one class in the inheritance:

In [ ]:
class MoldyInstantBurningForest(InstantBurnForest, MoldProneForest):
    @property
    def losses(self):
        return self.fires | self.mold

    def advance_one_step(self):
        self.grow_trees()
        self._grow_mold_and_kill_trees()
        self._strike_and_burn()

It's only a matter of defining the appropriate calculation for the `losses` attribute.  In this case we want the union of the fires and the mold sites.

Then we overwrite the critical `.advance_one_step()` method to grow the trees, then grow the mold and kill the trees, then strike and burn.  Our choice of order depends on how we want to model the physics of the forest.

In [ ]:
forest = MoldyInstantBurningForest()

In [ ]:
tree_history = []
forests = [
    Forest(),
    MoldProneForest(),
    InstantBurnForest(),
    MoldyInstantBurningForest(),
]
num_generations = 2500
for i in xrange(num_generations):
    for forest in forests:
        forest.advance_one_step()
    tree_history.append(tuple(f.tree_fraction for f in forests))

In [ ]:
f = figure(figsize=(10,5))
p = plot(tree_history)
l = legend([f.name for f in forests], loc="upper right")

Copyright 2008-2016, Enthought, Inc.<br>Use only permitted under license.  Copying, sharing, redistributing or other unauthorized use strictly prohibited.<br>http://www.enthought.com